# Georgia

In [1]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# load data
# data = pd.read_csv(r"C:\Users\georg\Documents\BPC project\steps_dataset\georgia\geo2016.csv")
data = pd.read_csv('../../steps_dataset/georgia/geo2016.csv')

/Users/ebisong/opt/anaconda3/envs/bpc/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (5,16,29,49,110,172,176,185) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
data.head()

,pid,c1,c4,c5,c6,c6other,c7,c8,c9,c10type,...,b15,b17,psu,stratum,age,agerange,wstep1,wstep2,wstep3,wstepurine
0,5,2,12,5,1,NaN,2,2,5,don't know,...,10.0,1.21,307,10,25,18-29,433.820007,433.820007,632.500000,1060.560059
1,6,1,12,6,1,NaN,2,8,2,month,...,10.0,1.05,133,11,60,60-69,173.169998,173.169998,209.240005,324.690002
2,7,1,13,6,1,NaN,2,8,2,don't know,...,10.0,0.76,6,3,64,60-69,359.290008,359.290008,434.140015,673.679993
3,8,2,11,5,1,NaN,2,6,4,month,...,10.0,0.53,365,4,54,45-59,121.339996,121.339996,148.559998,236.490005
4,9,1,10,4,1,NaN,1,9,1,month,...,10.0,0.73,204,9,60,60-69,199.699997,199.699997,241.300003,374.440002


In [4]:
print('Basic Information:')
print(f'Number of rows: {len(data)}')
print(f'Number of columns: {data.shape[1]}')

Basic Information:
Number of rows: 4204
Number of columns: 220


In [5]:
# create final dataframe - this dataframe is to be appended to as we create the new features
cleaned_data = pd.DataFrame()

In [6]:
# append country
country = ['georgia'] * data.shape[0]
country = pd.DataFrame({'country': country})
cleaned_data = pd.concat([cleaned_data, country], axis = 1)

In [7]:
# append sex - (category)
cleaned_data = pd.concat([cleaned_data, data['c1']], axis = 1)
cleaned_data.rename(columns={'c1':'sex'}, inplace=True)

In [8]:
# append age - (numeric)
cleaned_data = pd.concat([cleaned_data, data['age']], axis = 1)
cleaned_data['age'] = pd.to_numeric(cleaned_data['age'], errors='coerce')

In [9]:
# append years-at-school - (numeric)
cleaned_data = pd.concat([cleaned_data, data['c4']], axis = 1)
cleaned_data.rename(columns={'c4':'years-at-school'}, inplace=True)
cleaned_data['years-at-school'] = pd.to_numeric(cleaned_data['years-at-school'], errors='coerce')

In [10]:
# append level-of-education - (category)
cleaned_data = pd.concat([cleaned_data, data['c5']], axis = 1)
cleaned_data.rename(columns={'c5':'level-of-education'}, inplace=True)

In [11]:
# append marital-status - (category)
cleaned_data = pd.concat([cleaned_data, data['c7']], axis = 1)
cleaned_data.rename(columns={'c7':'marital-status'}, inplace=True)

In [12]:
# append work-status - (category)
cleaned_data = pd.concat([cleaned_data, data['c8']], axis = 1)
cleaned_data.rename(columns={'c8':'work-status'}, inplace=True)

In [13]:
# append ppl-in-household - (numeric)
cleaned_data = pd.concat([cleaned_data, data['c9']], axis = 1)
cleaned_data.rename(columns={'c9':'ppl-in-household'}, inplace=True)
cleaned_data['ppl-in-household'] = pd.to_numeric(cleaned_data['ppl-in-household'], errors='coerce')

In [14]:
# append earnings per year - (numeric)
earnings_per_year = []
for index, row in data.iterrows():
    if row['c10type'] == 'month':
        earnings_per_year.append(int(row['c10']) * 12)
    elif row['c10type'] == 'year':
        earnings_per_year.append(int(row['c10']))
    else:
        earnings_per_year.append(np.nan)
        
earnings_per_year = pd.DataFrame({'earnings-per-year': earnings_per_year})
cleaned_data = pd.concat([cleaned_data, earnings_per_year], axis = 1)
cleaned_data['earnings-per-year'] = pd.to_numeric(cleaned_data['earnings-per-year'], errors='coerce')

In [15]:
# append - Do you currently smoke tobacco products? - (category)
cleaned_data = pd.concat([cleaned_data, data['t1']], axis = 1)
cleaned_data.rename(columns={'t1':'currently-smoke-tobacco'}, inplace=True)

In [16]:
# append - How old were you when you first started smoking? - (numeric)
cleaned_data = pd.concat([cleaned_data, data['t3']], axis = 1)
cleaned_data.rename(columns={'t3':'age-started-smoking'}, inplace=True)
cleaned_data['age-started-smoking'] = pd.to_numeric(cleaned_data['age-started-smoking'], errors='coerce')

In [17]:
# append - Length of time smoking - (numeric)
try:
        length_time_smoking = []
        for index, row in data.iterrows():
            if row['t4c'].is_integer():
                if row['t4c'] == 77: # convert 77 (don't know) to np.nan
                    length_time_smoking.append(np.nan)
                else:
                    length_time_smoking.append(int(row['t4c']) * 52.1429) # weeks to year
            elif row['t4b'].is_integer():
                if row['t4b'] == 77: # convert 77 (don't know) to np.nan
                    length_time_smoking.append(np.nan)
                else:
                    length_time_smoking.append(int(row['t4b']) * 12) # months to year
            elif row['t4a'].is_integer():
                if row['t4a'] == 77: # convert 77 (don't know) to np.nan
                    length_time_smoking.append(np.nan)
                else:
                    length_time_smoking.append(int(row['t4a'])) # year
            else:
                length_time_smoking.append(np.nan)
        
        length_time_smoking = pd.DataFrame({'length-time-smoking': length_time_smoking})
        cleaned_data = pd.concat([cleaned_data, length_time_smoking], axis = 1)
        cleaned_data['length-time-smoking'] = pd.to_numeric(cleaned_data['length-time-smoking'], errors='coerce')
except KeyError:
    cleaned_data['length-time-smoking'] = np.nan

In [18]:
# append - Number of smoke/ smokeless per day (tobacco products) - (numeric)
# append - What type of tobacco do you use? - (category)
number_tobacco = []
type_tobacco = []

for index, row in data.iterrows():
    tobacco = []
    tobacco.append(int(row['t5a'])) if row['t5a'].is_integer() else  tobacco.append(0)
    tobacco.append(round(int(row['t5aw']) / 7, 0)) if row['t5aw'].is_integer() else  tobacco.append(0)
    tobacco.append(int(row['t5b'])) if row['t5b'].is_integer() else  tobacco.append(0)
    tobacco.append(round(int(row['t5bw']) / 7, 0)) if row['t5bw'].is_integer() else  tobacco.append(0)
    tobacco.append(int(row['t5c'])) if row['t5c'].is_integer() else  tobacco.append(0)
    tobacco.append(round(int(row['t5cw']) / 7, 0)) if row['t5cw'].is_integer() else  tobacco.append(0)
    tobacco.append(int(row['t5d'])) if row['t5d'].is_integer() else  tobacco.append(0)
    tobacco.append(round(int(row['t5dw']) / 7, 0)) if row['t5dw'].is_integer() else  tobacco.append(0)
    tobacco.append(int(row['t5e'])) if row['t5e'].is_integer() else  tobacco.append(0)
    tobacco.append(round(int(row['t5ew']) / 7, 0)) if row['t5ew'].is_integer() else  tobacco.append(0)
    tobacco.append(int(row['t5f'])) if row['t5f'].is_integer() else  tobacco.append(0)
    tobacco.append(round(int(row['t5fw']) / 7, 0)) if row['t5fw'].is_integer() else  tobacco.append(0)
    
    number_tobacco.append(max(tobacco)) if max(tobacco) != 0 else number_tobacco.append(np.nan)
    
    # type of tobacco
    index = tobacco.index(max(tobacco)) if max(tobacco) != 0 else 99
    
    if index == 0 or index == 1:
        type_tobacco.append('manufactured cigarettes')
    elif index == 2 or index == 3:
        type_tobacco.append('hand-rolled cigarettes')
    elif index == 4 or index == 5:
        type_tobacco.append('tobacco pipes')
    elif index == 6 or index == 7:
        type_tobacco.append('cigars, cheroots, cigarillos')
    elif index == 8 or index == 9:
        type_tobacco.append('shisha')
    elif index == 10 or index == 11:
        type_tobacco.append('other tobacco product')
    else:
        type_tobacco.append('none')
        
# append number of tobacco
number_tobacco = pd.DataFrame({'number-tobacco': number_tobacco})
cleaned_data = pd.concat([cleaned_data, number_tobacco], axis = 1)
cleaned_data['number-tobacco'] = pd.to_numeric(cleaned_data['number-tobacco'], errors='coerce')

# append type of robacco
type_tobacco = pd.DataFrame({'type-tobacco': type_tobacco})
cleaned_data = pd.concat([cleaned_data, type_tobacco], axis = 1)


In [19]:
# append - How old when you stopped smoking (weeks) - (numeric)
age_stopped_smoking = []
for index, row in data.iterrows():    
    if row['t10'].is_integer():
        age_stopped_smoking.append(int(row['t10'])) if int(row['t10']) != 77 else age_stopped_smoking.append(np.nan)
    else:
        age_stopped_smoking.append(np.nan)
            
        
age_stopped_smoking = pd.DataFrame({'age-stopped-smoking': age_stopped_smoking})
cleaned_data = pd.concat([cleaned_data, age_stopped_smoking], axis = 1)
cleaned_data['age-stopped-smoking'] = pd.to_numeric(cleaned_data['age-stopped-smoking'], errors='coerce')

In [20]:
# append - Smoke in home or workplace? - (category)
smoke_home_workplace = []
for index, row in data.iterrows():
    if row['t17'] or row['t17']:
        if int(row['t17']) == 1 or int(row['t18']) == 1:
            smoke_home_workplace.append(1)
        else:
            smoke_home_workplace.append(2)
    else:
        smoke_home_workplace.append(np.nan)
        
smoke_home_workplace = pd.DataFrame({'smoke-home-workplace': smoke_home_workplace})
cleaned_data = pd.concat([cleaned_data, smoke_home_workplace], axis = 1)

In [21]:
# append - How you ever consumed alcohol? - (category)
consumed_alcohol = []
for index, row in data.iterrows():
    if row['a1']:
        consumed_alcohol.append(int(row['a1']))
    else:
        consumed_alcohol.append(np.nan)
        
consumed_alcohol = pd.DataFrame({'consumed-alcohol': consumed_alcohol})
cleaned_data = pd.concat([cleaned_data, consumed_alcohol], axis = 1)

In [22]:
# append - Have you stopped drinking due to health reasons? - (category)
quit_drinking_for_health = []
for index, row in data.iterrows():
    if row['a3'].is_integer():
        quit_drinking_for_health.append(int(row['a3']))
    else:
        quit_drinking_for_health.append(np.nan)
        
quit_drinking_for_health = pd.DataFrame({'quit-drinking-for-health': quit_drinking_for_health})
cleaned_data = pd.concat([cleaned_data, quit_drinking_for_health], axis = 1)

In [23]:
# append - How many alcoholic drinks do you consume per day? - (category)
number_alcoholic_drinks = []
for index, row in data.iterrows():
    if row['a4'].is_integer():
        number_alcoholic_drinks.append(int(row['a4']))
    else:
        number_alcoholic_drinks.append(np.nan)
        
number_alcoholic_drinks = pd.DataFrame({'number-alcoholic-drinks': number_alcoholic_drinks})
cleaned_data = pd.concat([cleaned_data, number_alcoholic_drinks], axis = 1)

In [24]:
# append - How many fruit/ vegetables do you eat per day? - (numeric)
number_daily_fruit_vegetables = []
for index, row in data.iterrows():
    fruit = 0
    vegetable = 0
    
    # fruit
    if row['d1']:
        if row['d2']:
            fruit = (row['d1']) * (row['d2'])
    else:
            fruit = 0
    
    # vegetables
    if row['d3']:
        if row['d4']:
            vegetable = (row['d3']) * (row['d4'])
    else:
            vegetable = 0
            
    number_daily_fruit_vegetables.append(fruit + vegetable)
        
number_daily_fruit_vegetables = pd.DataFrame({'number-daily-fruit-vegetables': number_daily_fruit_vegetables})
cleaned_data = pd.concat([cleaned_data, number_daily_fruit_vegetables], axis = 1)

In [25]:
# append - How much salt or salty sauce do you think you consume?  - (category)
cleaned_data = pd.concat([cleaned_data, data['d8']], axis = 1)
cleaned_data.rename(columns={'d8':'salt-consumption'}, inplace=True)

In [26]:
# append - How intense is your work [vigorous- intensity, moderate-intensity, none]? - (cateogry)
work_intensity = []
for index, row in data.iterrows():
    if row['p1'] or row['p4']:
        if row['p1'] and int(row['p1']) == 1:
            work_intensity.append('vigorous-intensity')
        elif row['p4'] and int(row['p4']) == 1 and int(row['p1']) != 1:
            work_intensity.append('moderate-intensity')
        else:
             work_intensity.append('none')
    else:
        work_intensity.append(np.nan)
        
work_intensity = pd.DataFrame({'work-intensity': work_intensity})
cleaned_data = pd.concat([cleaned_data, work_intensity], axis = 1)

In [27]:
# append - In a typical week, on how many days do you do moderate or vigorous-intensity activities as part of your work? - (numeric)
days_vigorous_exercise = []
cleaned_data = pd.concat([cleaned_data, data['p2']], axis = 1)
cleaned_data.rename(columns={'p2':'days-vigorous-exercise'}, inplace=True)
cleaned_data['days-vigorous-exercise'] = pd.to_numeric(cleaned_data['days-vigorous-exercise'], errors='coerce')

days_moderate_exercise = []
cleaned_data = pd.concat([cleaned_data, data['p5']], axis = 1)
cleaned_data.rename(columns={'p5':'days-moderate-exercise'}, inplace=True)
cleaned_data['days-moderate-exercise'] = pd.to_numeric(cleaned_data['days-moderate-exercise'], errors='coerce')

In [28]:
# append - How much time do you spend walking or bicycling for travel on a typical day? - (numeric)
time_walking_bicycling_minutes = []

for index, row in data.iterrows():
    if row['p9a'].is_integer() or row['p9b'].is_integer():
        hour_to_minute = minute = 0
         
        if row['p9a'].is_integer():
            hour_to_minute = int(row['p9a']) * 60
        elif row['p9b'].is_integer():
            minute = int(row['p9b'])
        
        time_walking_bicycling_minutes.append(hour_to_minute + minute)
    else:
        time_walking_bicycling_minutes.append(np.nan)
        
time_walking_bicycling_minutes = pd.DataFrame({'time-walking-bicycling-minutes': time_walking_bicycling_minutes})
cleaned_data = pd.concat([cleaned_data, time_walking_bicycling_minutes], axis = 1)

In [29]:
# append - How much time do you usually spend sitting or reclining on a typical day? - (numeric)
time_sedentary = []

for index, row in data.iterrows():
    if row['p16a'] or row['p16b']:
        hour_to_minute = minute = 0
         
        if row['p16a']:
            hour_to_minute = int(row['p16a']) * 60
        elif row['p16b']:
            minute = int(row['p16b'])
        
        time_sedentary.append(hour_to_minute + minute)
    else:
        time_sedentary.append(np.nan)
        
time_sedentary = pd.DataFrame({'time-sedentary': time_sedentary})
cleaned_data = pd.concat([cleaned_data, time_sedentary], axis = 1)

In [30]:
# append - Have you ever had your blood pressure measured by a doctor or other health worker? - (category)
cleaned_data = pd.concat([cleaned_data, data['h1']], axis = 1)
cleaned_data.rename(columns={'h1':'had-blood-pressure-measurement'}, inplace=True)

In [31]:
# append - Have you taken any drugs (medication) for raised blood pressure? - (category)
cleaned_data = pd.concat([cleaned_data, data['h3']], axis = 1)
cleaned_data.rename(columns={'h3':'taken-drugs-for-raised-bp'}, inplace=True)
cleaned_data['taken-drugs-for-raised-bp'] = pd.to_numeric(cleaned_data['taken-drugs-for-raised-bp'], errors='coerce')

In [32]:
# append - Have you ever had your blood sugar measured by a doctor or other health worker? - (category)
cleaned_data = pd.concat([cleaned_data, data['h6']], axis = 1)
cleaned_data.rename(columns={'h6':'had-blood-sugar-measurement'}, inplace=True)
cleaned_data['had-blood-sugar-measurement'] = pd.to_numeric(cleaned_data['had-blood-sugar-measurement'], errors='coerce')

In [33]:
# append - Have you taken any drugs (medication) for diabetes? - (category)
cleaned_data = pd.concat([cleaned_data, data['h8']], axis = 1)
cleaned_data.rename(columns={'h8':'taken-diabetes-drugs'}, inplace=True)
cleaned_data['taken-diabetes-drugs'] = pd.to_numeric(cleaned_data['taken-diabetes-drugs'], errors='coerce')

In [34]:
# append - Have you ever had your cholesterol (fat levels in your blood) measured by a doctor or other health worker? - (category)
cleaned_data = pd.concat([cleaned_data, data['h12']], axis = 1)
cleaned_data.rename(columns={'h12':'had-cholesterol-measurement'}, inplace=True)
cleaned_data['had-cholesterol-measurement'] = pd.to_numeric(cleaned_data['had-cholesterol-measurement'], errors='coerce')

In [35]:
# append - Have you taken any oral treatment (medication) for raised total cholesterol? - (category)
cleaned_data = pd.concat([cleaned_data, data['h14']], axis = 1)
cleaned_data.rename(columns={'h14':'taken-cholesterol-oral-treatment'}, inplace=True)
cleaned_data['taken-cholesterol-oral-treatment'] = pd.to_numeric(cleaned_data['taken-cholesterol-oral-treatment'], errors='coerce')

In [36]:
# append - Have you ever had a heart attack or chest pain from heart disease (angina) or a stroke (cerebrovascular accident or incident)? - (category)
cleaned_data = pd.concat([cleaned_data, data['h17']], axis = 1)
cleaned_data.rename(columns={'h17':'had-heart-attack'}, inplace=True)
cleaned_data['had-heart-attack'] = pd.to_numeric(cleaned_data['had-heart-attack'], errors='coerce')

In [37]:
# append - Are you currently taking medication to prevent or treat heart disease? - (category)
cleaned_data = pd.concat([cleaned_data, data['h18']], axis = 1)
cleaned_data.rename(columns={'h18':'taking-heart-disease-medication'}, inplace=True)
cleaned_data['taking-heart-disease-medication'] = pd.to_numeric(cleaned_data['taking-heart-disease-medication'], errors='coerce')

In [38]:
# append - Reading 1 [Systolic, Diastolic] - (numeric)

# systolic
cleaned_data = pd.concat([cleaned_data, data['m4a']], axis = 1)
cleaned_data.rename(columns={'m4a':'reading1-systolic'}, inplace=True)
cleaned_data['reading1-systolic'] = pd.to_numeric(cleaned_data['reading1-systolic'], errors='coerce')

# diastolic
cleaned_data = pd.concat([cleaned_data, data['m4b']], axis = 1)
cleaned_data.rename(columns={'m4b':'reading1-diastolic'}, inplace=True)
cleaned_data['reading1-diastolic'] = pd.to_numeric(cleaned_data['reading1-diastolic'], errors='coerce')

In [39]:
# append - Reading 2 [Systolic, Diastolic] - (numeric)

# systolic
cleaned_data = pd.concat([cleaned_data, data['m5a']], axis = 1)
cleaned_data.rename(columns={'m5a':'reading2-systolic'}, inplace=True)
cleaned_data['reading2-systolic'] = pd.to_numeric(cleaned_data['reading2-systolic'], errors='coerce')

# diastolic
cleaned_data = pd.concat([cleaned_data, data['m5b']], axis = 1)
cleaned_data.rename(columns={'m5b':'reading2-diastolic'}, inplace=True)
cleaned_data['reading2-diastolic'] = pd.to_numeric(cleaned_data['reading2-diastolic'], errors='coerce')

In [40]:
# append - Reading 3 [Systolic, Diastolic] - (numeric)

# systolic
cleaned_data = pd.concat([cleaned_data, data['m6a']], axis = 1)
cleaned_data.rename(columns={'m6a':'reading3-systolic'}, inplace=True)
cleaned_data['reading3-systolic'] = pd.to_numeric(cleaned_data['reading3-systolic'], errors='coerce')

# diastolic
cleaned_data = pd.concat([cleaned_data, data['m6b']], axis = 1)
cleaned_data.rename(columns={'m6b':'reading3-diastolic'}, inplace=True)
cleaned_data['reading3-diastolic'] = pd.to_numeric(cleaned_data['reading3-diastolic'], errors='coerce')

In [41]:
# append - Have you been treated for raised blood pressure? - (category)
cleaned_data = pd.concat([cleaned_data, data['m7']], axis = 1)
cleaned_data.rename(columns={'m7':'treated-for-raised-bp'}, inplace=True)
cleaned_data['treated-for-raised-bp'] = pd.to_numeric(cleaned_data['treated-for-raised-bp'], errors='coerce')

In [42]:
# append - For women: Are you pregnant? - (category)
cleaned_data = pd.concat([cleaned_data, data['m8']], axis = 1)
cleaned_data.rename(columns={'m8':'are-you-pregnant'}, inplace=True)
cleaned_data['are-you-pregnant'] = pd.to_numeric(cleaned_data['are-you-pregnant'], errors='coerce')

In [43]:
# append - Height - (numeric)
try:
    cleaned_data = pd.concat([cleaned_data, data['m11']], axis = 1)
    cleaned_data.rename(columns={'m11':'height'}, inplace=True)
    cleaned_data['height'] = pd.to_numeric(cleaned_data['height'], errors='coerce')
except KeyError:
    cleaned_data['height'] = np.nan

In [44]:
# append - Weight - (numeric)
try:
    cleaned_data = pd.concat([cleaned_data, data['m12']], axis = 1)
    cleaned_data.rename(columns={'m12':'weight'}, inplace=True)
    cleaned_data['weight'] = pd.to_numeric(cleaned_data['weight'], errors='coerce')
except KeyError:
    cleaned_data['weight'] = np.nan

In [45]:
# append - Waist circumference - (numeric)
try:
    cleaned_data = pd.concat([cleaned_data, data['m14']], axis = 1)
    cleaned_data.rename(columns={'m14':'waist-circumference'}, inplace=True)
    cleaned_data['waist-circumference'] = pd.to_numeric(cleaned_data['waist-circumference'], errors='coerce')
except KeyError:
    cleaned_data['waist-circumference'] = np.nan

In [46]:
# append - hip circumference - (numeric)
try:
    cleaned_data = pd.concat([cleaned_data, data['m15']], axis = 1)
    cleaned_data.rename(columns={'m15':'hip-circumference'}, inplace=True)
    cleaned_data['hip-circumference'] = pd.to_numeric(cleaned_data['hip-circumference'], errors='coerce')
except KeyError:
    cleaned_data['hip-circumference'] = np.nan

In [47]:
# append - Reading 1 (beats per minute) - (numeric)
try:
    cleaned_data = pd.concat([cleaned_data, data['m16a']], axis = 1)
    cleaned_data.rename(columns={'m16a':'reading1-bpm'}, inplace=True)
    cleaned_data['reading1-bpm'] = pd.to_numeric(cleaned_data['reading1-bpm'], errors='coerce')
except KeyError:
    cleaned_data['reading1-bpm'] = np.nan

In [48]:
# append - Reading 2 (beats per minute) - (numeric)
try:
    cleaned_data = pd.concat([cleaned_data, data['m16b']], axis = 1)
    cleaned_data.rename(columns={'m16b':'reading2-bpm'}, inplace=True)
    cleaned_data['reading2-bpm'] = pd.to_numeric(cleaned_data['reading2-bpm'], errors='coerce')
except KeyError:
    cleaned_data['reading2-bpm'] = np.nan

In [49]:
# append - Reading 3 (beats per minute) - (numeric)
try:
    cleaned_data = pd.concat([cleaned_data, data['m16c']], axis = 1)
    cleaned_data.rename(columns={'m16c':'reading3-bpm'}, inplace=True)
    cleaned_data['reading3-bpm'] = pd.to_numeric(cleaned_data['reading3-bpm'], errors='coerce')
except KeyError:
    cleaned_data['reading3-bpm'] = np.nan

In [50]:
# append - Fasting blood glucose - (numeric)
try:
    cleaned_data = pd.concat([cleaned_data, data['b5']], axis = 1)
    cleaned_data.rename(columns={'b5':'fasting-blood-glucose'}, inplace=True)
    cleaned_data['fasting-blood-glucose'] = pd.to_numeric(cleaned_data['fasting-blood-glucose'], errors='coerce')
except KeyError:
    cleaned_data['fasting-blood-glucose'] = np.nan

In [51]:
# append - Total cholesterol - (numeric)
try:
    cleaned_data = pd.concat([cleaned_data, data['b8']], axis = 1)
    cleaned_data.rename(columns={'b8':'total-cholesterol'}, inplace=True)
    cleaned_data['total-cholesterol'] = pd.to_numeric(cleaned_data['total-cholesterol'], errors='coerce')
except KeyError:
    cleaned_data['total-cholesterol'] = np.nan

In [52]:
# append - Urinary sodium - (numeric)
try:
    cleaned_data = pd.concat([cleaned_data, data['b14']], axis = 1)
    cleaned_data.rename(columns={'b14':'urinary-sodium'}, inplace=True)
    cleaned_data['urinary-sodium'] = pd.to_numeric(cleaned_data['urinary-sodium'], errors='coerce')
except KeyError:
    cleaned_data['urinary-sodium'] = np.nan

In [53]:
# append - Urinary creatinine - (numeric)
try:
    cleaned_data = pd.concat([cleaned_data, data['b15']], axis = 1)
    cleaned_data.rename(columns={'b15':'urinary-creatinine'}, inplace=True)
    cleaned_data['urinary-creatinine'] = pd.to_numeric(cleaned_data['urinary-creatinine'], errors='coerce')
except KeyError:
    cleaned_data['urinary-creatinine'] = np.nan

In [54]:
# append - Triglycerides - (numeric)
try:
    cleaned_data = pd.concat([cleaned_data, data['b16']], axis = 1)
    cleaned_data.rename(columns={'b16':'triglycerides'}, inplace=True)
    cleaned_data['triglycerides'] = pd.to_numeric(cleaned_data['triglycerides'], errors='coerce')
except KeyError:
    cleaned_data['triglycerides'] = np.nan

In [55]:
# append - Urinary creatinine - (numeric)
try:
    cleaned_data = pd.concat([cleaned_data, data['b17']], axis = 1)
    cleaned_data.rename(columns={'b17':'hdl-cholesterol'}, inplace=True)
    cleaned_data['hdl-cholesterol'] = pd.to_numeric(cleaned_data['hdl-cholesterol'], errors='coerce')
except KeyError:
    cleaned_data['hdl-cholesterol'] = np.nan

In [56]:
cleaned_data.head(20)
# cleaned_data.tail(20)

,country,sex,age,years-at-school,level-of-education,marital-status,work-status,ppl-in-household,earnings-per-year,currently-smoke-tobacco,...,hip-circumference,reading1-bpm,reading2-bpm,reading3-bpm,fasting-blood-glucose,total-cholesterol,urinary-sodium,urinary-creatinine,triglycerides,hdl-cholesterol
0,georgia,2,25,12,5,2,2,5,NaN,2,...,100.0,72.0,72.0,70.0,3.900000,6.41,69.199997,10.0,NaN,1.21
1,georgia,1,60,12,6,2,8,2,7200.0,2,...,122.0,80.0,79.0,78.0,4.600000,4.67,69.800003,10.0,NaN,1.05
2,georgia,1,64,13,6,2,8,2,NaN,2,...,105.0,93.0,98.0,94.0,19.200001,5.38,70.900002,10.0,NaN,0.76
3,georgia,2,54,11,5,2,6,4,2400.0,2,...,107.0,91.0,87.0,93.0,6.100000,3.85,71.599998,10.0,NaN,0.53
4,georgia,1,60,10,4,1,9,1,2400.0,2,...,120.0,99.0,93.0,91.0,8.400000,3.07,64.699997,10.0,NaN,0.73
5,georgia,2,51,15,7,2,6,3,1620.0,2,...,123.0,77.0,76.0,77.0,6.700000,6.22,65.599998,10.0,NaN,0.92
6,georgia,2,58,10,4,2,6,1,NaN,2,...,79.0,69.0,73.0,67.0,1.100000,2.59,117.000000,10.0,NaN,1.58
7,georgia,1,33,10,7,2,3,1,7200.0,2,...,101.0,75.0,72.0,76.0,3.700000,3.89,123.199997,10.0,NaN,0.62
8,georgia,1,65,10,4,2,7,2,2160.0,2,...,111.0,69.0,60.0,68.0,4.700000,3.99,124.900002,10.0,NaN,0.77
9,georgia,2,54,16,7,2,6,2,NaN,1,...,112.0,72.0,72.0,72.0,3.900000,6.50,127.599998,10.0,NaN,0.88


In [57]:
cleaned_data.shape

(4204, 55)

In [58]:
cleaned_data.columns

Index(['country', 'sex', 'age', 'years-at-school', 'level-of-education',
       'marital-status', 'work-status', 'ppl-in-household',
       'earnings-per-year', 'currently-smoke-tobacco', 'age-started-smoking',
       'length-time-smoking', 'number-tobacco', 'type-tobacco',
       'age-stopped-smoking', 'smoke-home-workplace', 'consumed-alcohol',
       'quit-drinking-for-health', 'number-alcoholic-drinks',
       'number-daily-fruit-vegetables', 'salt-consumption', 'work-intensity',
       'days-vigorous-exercise', 'days-moderate-exercise',
       'time-walking-bicycling-minutes', 'time-sedentary',
       'had-blood-pressure-measurement', 'taken-drugs-for-raised-bp',
       'had-blood-sugar-measurement', 'taken-diabetes-drugs',
       'had-cholesterol-measurement', 'taken-cholesterol-oral-treatment',
       'had-heart-attack', 'taking-heart-disease-medication',
       'reading1-systolic', 'reading1-diastolic', 'reading2-systolic',
       'reading2-diastolic', 'reading3-systolic', 'rea

### Export cleaned data as csv

In [59]:
# cleaned_data.to_csv('C:\\Users\\georg\\Documents\\BPC Cleaned data\\georgia.csv', index=False, encoding="utf-8")
cleaned_data.to_csv('../../cleaned_data/georgia.csv', encoding='utf-8', index=False)

# End